In [41]:
from functools import reduce
import re
import pandas as pd
import numpy as np
from scipy.spatial.distance import mahalanobis
from scipy.optimize import linear_sum_assignment
import warnings

warnings.filterwarnings('ignore')

ЗДЕСЬ И СОБЫТИЯ ЯНДЕКСА И РОССТАТ

In [ ]:
events1 = pd.read_csv('события.csv')
events2 = pd.read_csv('росстат_общий_new1.csv')


df = events2.merge(
    events1[['Event Text', 'region_name', 'municipality', 'year']],
    on=['municipality', 'region_name', 'year'],
    how='left'
)


df.to_csv('общие_данные_2012t+.csv', index=False)
df

,year,region_name,municipality,oktmo,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),Население,Земля,Расходы,ПотреблениеВ,ПотреблениеТ,ПотреблениеЭ,БезДорог,Прибыль,Бюджет,Квартиры,Зарплата,Event Text
0,2006,Тверская,NaN,28602100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-433.0,0.0,0.0,NaN
1,2006,Тверская,NaN,28602400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,284.0,0.0,0.0,NaN
2,2006,Тверская,NaN,28604100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-375.0,0.0,0.0,NaN
3,2006,Тверская,NaN,28604400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,345.0,0.0,0.0,NaN
4,2006,Тверская,NaN,28606100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519659,2024,Санкт-Петербург,№,40322000,0.0,0.0,0.0,0.0,191571.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
519660,2024,Санкт-Петербург,№,40331000,0.0,0.0,0.0,0.0,76567.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
519661,2024,Санкт-Петербург,№,40383000,0.0,0.0,0.0,0.0,67469.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
519662,2024,Санкт-Петербург,№,40903000,0.0,0.0,0.0,0.0,67637.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


ОБЪЕДИНЕНИЕ ПРОТЕСТОВ С ДАННЫМИ РОССТАТА 

1. ВООБЩЕ 
2. ТОЛЬКО ЗА 2012

In [ ]:
data = pd.read_csv('общие_данные_2012t+.csv')

protest_events = ["Demonstrate or rally", "Criticize or denounce", "Defy norms, law"]
protests = data[data['Event Text'].isin(protest_events)]
protests.to_csv('протесты_2012t+.csv', index=False)

protests1 = protests[protests['year'].isin([2011, 2012, 2013])].reset_index(drop=True)
protests1 = protests1.drop_duplicates(subset=['oktmo']).reset_index(drop=True)
protests1.to_csv('протесты2012_2012t+.csv', index=False)
protests

,year,region_name,municipality,oktmo,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),Население,Земля,Расходы,ПотреблениеВ,ПотреблениеТ,ПотреблениеЭ,БезДорог,Прибыль,Бюджет,Квартиры,Зарплата,Event Text
245,2006,Татарстан,Агрызский,92601000,0.0,0.0,0.0,0.0,0.0,179662.2,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,Criticize or denounce
246,2006,Татарстан,Агрызский,92601000,0.0,0.0,0.0,0.0,0.0,179662.2,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,Criticize or denounce
282,2006,Татарстан,Агрызский,92601000,0.0,0.0,0.0,0.0,0.0,179662.2,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,Criticize or denounce
294,2006,Татарстан,Агрызский,92601000,0.0,0.0,0.0,0.0,0.0,179662.2,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,Demonstrate or rally
348,2006,Татарстан,Агрызский,92601000,0.0,0.0,0.0,0.0,0.0,179662.2,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,Criticize or denounce
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504660,2023,Хабаровский,Хабаровский,8655000,0.0,61.9,0.0,0.0,82008.0,0.0,0.0,18.7,0.22,1333.50,2.71,0.0,0.0,0.0,67779.7,Demonstrate or rally
504662,2023,Хабаровский,Хабаровский,8655000,0.0,61.9,0.0,0.0,82008.0,0.0,0.0,18.7,0.22,1333.50,2.71,0.0,0.0,0.0,67779.7,Demonstrate or rally
504665,2023,Хабаровский,Хабаровский,8655000,0.0,61.9,0.0,0.0,82008.0,0.0,0.0,18.7,0.22,1333.50,2.71,0.0,0.0,0.0,67779.7,Demonstrate or rally
504667,2023,Хабаровский,Хабаровский,8655000,0.0,61.9,0.0,0.0,82008.0,0.0,0.0,18.7,0.22,1333.50,2.71,0.0,0.0,0.0,67779.7,Demonstrate or rally


ЕДИНИЦЫ БЕЗ ПРОТЕСТОВ СРЕДИ ВСЕХ СОБЫТИЙ ЗА ВСЕ ВРЕМЯ

In [ ]:
lol = pd.read_csv("протесты_2012t+.csv") 
sobytia = pd.read_csv("общие_данные_2012t+.csv")


noprotestALL = sobytia[~sobytia['municipality'].isin(lol['municipality'])].reset_index(drop=True)
noprotestALL.to_csv('единицы_без_протестов_2012t+.csv', index=False)
noprotestALL

,year,region_name,municipality,oktmo,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),Население,Земля,Расходы,ПотреблениеВ,ПотреблениеТ,ПотреблениеЭ,БезДорог,Прибыль,Бюджет,Квартиры,Зарплата,Event Text
0,2006,Тверская,NaN,28602100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-433.0,0.0,0.0,NaN
1,2006,Тверская,NaN,28602400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,284.0,0.0,0.0,NaN
2,2006,Тверская,NaN,28604100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-375.0,0.0,0.0,NaN
3,2006,Тверская,NaN,28604400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,345.0,0.0,0.0,NaN
4,2006,Тверская,NaN,28606100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418325,2024,Санкт-Петербург,№,40322000,0.0,0.0,0.0,0.0,191571.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
418326,2024,Санкт-Петербург,№,40331000,0.0,0.0,0.0,0.0,76567.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
418327,2024,Санкт-Петербург,№,40383000,0.0,0.0,0.0,0.0,67469.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
418328,2024,Санкт-Петербург,№,40903000,0.0,0.0,0.0,0.0,67637.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


ПОИСК КОНТРОЛЬНОЙ ГРУППЫ (НЕПРОТЕСТНЫЕ ЕДИНИЦЫ ЗА 2011 ИЗ ТЕХ ЧТО ПРОТЕСТНЫЕ В 2012)

In [ ]:
lol = pd.read_csv("протесты2012_2012t+.csv") 
sobytia = pd.read_csv("общие_данные_2012t+.csv")

df_2011 = sobytia[sobytia['year'] == 2010]


filtered_df = df_2011[df_2011['oktmo'].isin(lol['oktmo'])]

pattern = "Demonstrate or rally|Criticize or denounce|Defy norms, law"
mask = filtered_df['Event Text'].str.contains(pattern, case=False, na=False)
municipalities_with_tag = filtered_df[mask]['oktmo'].unique()


final_df = filtered_df[~filtered_df['oktmo'].isin(municipalities_with_tag)].reset_index(drop=True)

final_df = final_df.drop_duplicates(subset='oktmo').reset_index(drop=True)
final_df.to_csv('без_протестов2011_2012t+.csv', index=False)
final_df



,year,region_name,municipality,oktmo,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),Население,Земля,Расходы,ПотреблениеВ,ПотреблениеТ,ПотреблениеЭ,БезДорог,Прибыль,Бюджет,Квартиры,Зарплата,Event Text
0,2010,Карачаево-Черкесская,Абазинский,91601000,0.00,0.00,0.0,3800.0,13280.0,30019.0,349335.4,0.00,0.00,0.00,23.30,0.0,-1356.2,230.00,10943.0,NaN
1,2010,Удмуртская,Алнашский,94602000,22.40,0.00,0.0,660.0,20718.0,89600.0,1781714.0,0.00,0.27,624.00,19.52,77987.0,3398.0,360.00,8804.1,Investigate
2,2010,Мурманская,Апатиты,47705000,1.40,0.00,300.0,500.0,60003.0,247136.0,3649634.0,43.00,0.25,855.00,0.00,26139.0,52509.0,510.80,25817.6,NaN
3,2010,Башкортостан,Архангельский,80603000,6.20,0.00,0.0,0.0,19829.0,242145.0,891669.0,0.00,0.32,928.00,5.10,-1269.0,8468.0,483.00,11085.9,NaN
4,2010,Башкортостан,Архангельский,80603410,0.00,0.00,0.0,0.0,0.0,6347.0,0.0,0.00,0.00,0.00,0.00,0.0,-292.0,0.00,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,2010,Калмыкия,Элиста,85701000,51.30,0.00,0.0,2963.0,106966.0,39657.0,3196832.7,6.50,0.62,442.70,0.95,-259896.0,-86985.4,323.00,14293.4,Reject
99,2010,Сахалинская,Южно-Сахалинск,64701000,171.00,1.03,0.0,4700.0,0.0,89827.0,26361579.0,29.20,0.25,3197.24,0.20,5061340.0,-427701.0,389.08,43408.3,Consult
100,2010,Саха,Якутск,98701000,311.76,0.00,1800.0,1300.0,278691.0,355334.0,21300345.4,41.24,0.40,1100.00,0.00,-686073.0,1380480.0,319.00,32882.1,"Arrest, detain, or charge with legal action"
101,2010,Башкортостан,Янаульский,80659000,10.70,0.00,200.0,5000.0,48916.0,209381.0,1967678.0,19.50,0.20,684.40,90.60,15548.0,2967.0,432.00,17995.5,Make statement


Создание регрессии

In [ ]:
def check_years_completeness(oktmo, df_events_full, required_years):
    municipality_data = df_events_full[df_events_full['oktmo'] == oktmo]
    existing_years = set(municipality_data['year'].unique())
    return existing_years.issuperset(required_years)

df_protests = pd.read_csv("протесты2012_2012t+.csv")
df_protests = df_protests.drop_duplicates(subset=['region_name', 'municipality'])
df_events = pd.read_csv("единицы_без_протестов_2012t+.csv")
df_events_full = pd.read_csv("общие_данные_2012t+.csv")

required_years = {2010.0, 2014.0, 2015.0, 2016.0}


df_events = df_events[df_events['year'].isin([2011, 2012, 2013])].reset_index(drop=True)

features = [
            'ПотреблениеЭ', 
            'ПотреблениеВ', 
            'Бюджет',
            'БезДорог', 
            'Земля',
            'Квартиры',  
            
            # 'Население', 
            # 'Прибыль',
            # 'Зарплата',
            # 'Расходы',

            # 'СреднегодН',
            # 'ПотреблениеТ', 
            # 'ЭконАктив'
            # 'Прирост', 
            # 'Рождаемость',
            # 'Смертность',
            # 'Родившихся',
            # 'Инвестиции',
]

df_protests = df_protests[
    ['region_name', 'municipality', 'oktmo', "Жилье (1000 м2)", "Дороги (%)", "Канализация (1 м)",
    "Водопровод (1 метр)"] + features
].reset_index(drop=True)
df_events = df_events[
    ['region_name', 'municipality', 'oktmo', "Жилье (1000 м2)", "Дороги (%)", "Канализация (1 м)",
    "Водопровод (1 метр)"] + features
].reset_index(drop=True)

exclude_dict = {
    'Абакан': ['Усть-Абаканский', 'Алтайский', 'Минусинский', 'Минусинск', 'Черногорск'],
    'Анапа': ['Темрюкский', 'Крымский', 'Новороссийск'],
    'Архангельск': ['Северодвинск', 'Приморский'],
    'Архаринский': ['Облученский', 'Верхнебуреинский', 'Бурейский'],
    'Бабаюрт': ['Шелковской', 'Хасавюртовский', 'Кизилюртовский', 'Кумторкалинский', 'Кизлярский'],
    'Балашиха': ['Королев', 'Люберцы', 'Богородский', 'Щелково', 'Раменский'],
    'Благовещенск': ['Благовещенский', 'Кушнаревский', 'Имангуловский'],
    'Бокситогорский': ['Тихвинский', 'Любытинский', 'Хвойнинский', 'Чагодощенский', 'Бабаевский'],
    'Брянск': ['Брянский', 'Сельцо', 'Фокино', 'Титовское'],
    'Варгашинский': ['Кетовский', 'Курган', 'Половинский', 'Лебяжьевский', 'Мокроусовский', 'Упоровский', 'Белозерский'],
    'Вологда': ['Вологодский'],
    'Воронеж': ['Хохольский', 'Семилукский', 'Рамонский', 'Новоусманский', 'Каширский'],
    'Гатчинский': ['Лужский', 'Волосовский', 'Тосненский', 'Пушкин', 'Ломоносовский'],
    'Геленджик': ['Туапсинский', 'Северский', 'Абинский', 'Крымский', 'Новороссийск'],
    'Горно-Алтайск': ['Майминский'],
    'Грязовецкий': ['Вологодский', 'Междуреченский', 'Солигаличский', 'Буйский', 'Любимский', 'Первомайский', 'Пошехонский'],
    'Гулькевичский': ['Кавказский', 'Новоалександровский', 'Новокубанский', 'Курганинский', 'Тбилисский'],
    'Гурьевский': ['Зеленоградский', 'Калининград', 'Полесский', 'Гвардейский', 'Багратионовский', 'Ладушкинский'],
    'Добрянский': ['Краснокамский', 'Ильинский', 'Юсьвинский', 'Березники', 'Александровский', 'Губаха', 'Чусовской', 'Пермский'],
    'Домодедово': ['Подольск', 'Ленинский', 'Раменский', 'Чехов', 'Ступино', 'Щербинка'],
    'Дубна': ['Кимрский', 'Кимры', 'Талдомский', 'Дмитровский', 'Конаковский'],
    'Завьяловский': ['Ижевск', 'Увинский', 'Малопургинский', 'Сарапульский', 'Чайковский', 'Воткинский', 'Якшур-Бодьинский'],
    'Ивановский': ['Иваново', 'Фурмановский', 'Комсомольский', 'Тейковский', 'Лежневский', 'Шуйский', 'Родниковский'],
    'Иркутск': ['Иркутский', 'Ангарское', 'Ангарск', 'Шелеховский'],
    'Казань': ['Лаишевский', 'Пестречинский', 'Высокогорский', 'Зеленодольский', 'Верхнеуслонский', 'Новоильмовское'],
    'Калуга': ['Перемышльский', 'Ферзиковский', 'Малоярославецкий', 'Дзержинский', 'Бабынинский'],
    'Кингисеппский': ['Сланцевский', 'Волосовский', 'Ломоносовский', 'Сосновоборский'],
    'Кинельский': ['Кинель', 'Волжский', 'Красноярский', 'Кинель-Черкасский', 'Богатовский', 'Нефтегорский', 'Приволжье'],
    'Кола': ['Североморск', 'Александровск', 'Кольский', 'Кильдинстрой'],
    'Комсомольск-на-Амуре': ['Комсомольский'],
    'Краснодар': ['Динский', 'Красноармейский', 'Тахтамукайский'],
    'Красноярский': ['Володарский', 'Приволжский', 'Наримановский', 'Харабалинский'],
    'Курганинский': ['Лабинский', 'Новокубанский', 'Гулькевичский', 'Тбилисский', 'Шовгеновский', 'Кошехабльский'],
    'Макаровский': ['Долинский', 'Томаринский', 'Углегорский', 'Поронайский', 'Вахрушев'],
    'Мезенский': ['Заполярный', 'Усть-Цилемский', 'Лешуконский', 'Пинежский', 'Приморский'],
    'Михайловский': ['Новомосковск', 'Кимовский', 'Скопинский', 'Пронский', 'Захаровский', 'Серебряные Пруды'],
    'Новороссийск': ['Новороссийский'],
    'Одинцовский': ['Власиха', 'Внуковское', 'Кокошкинское', 'Можайский', 'Красногорск', 'Марушкинское', 'Наро-Фоминский', 'Рузский', 'Истра'],
    'Омск': ['Омский'],
    'Оренбург': ['Оренбургский', 'Сакмарский'],
    'Орёл': ['Орловский'],
    'Петропавловск-Камчатский': ['Вилючинский', 'Елизовский'],
    'Пинежский': ['Лешуконский', 'Мезенский', 'Приморский', 'Холмогорский', 'Виноградовский', 'Верхнетоемский', 'Удорский'],
    'Почепский': ['Трубчевский', 'Погарский', 'Унечский', 'Мглинский', 'Клетнянский', 'Жирятинский', 'Выгоничский', 'Титовское'],
    'Пушкинский': ['Мытищи', 'Дмитровский', 'Сергиево-Посадский', 'Красноармейск', 'Щёлково', 'Восточный', 'Ивантеевка'],
    'Ростов': ['Ростовский'],
    'Саратовский': ['Красноармейский', 'Саратов', 'Лысогорский', 'Татищевский'],
    'Северодвинск': ['Приморский', 'Архангельск'],
    'Северский': ['Туапсинский', 'Геленджик', 'Абинский', 'Горячий Ключ', 'Тахтамукайский', 'Красноармейский'],
    'Тамбов': ['Тамбовский', 'Котовск'],
    'Таштыпский': ['Абаза', 'Таштагольский', 'Турочакский', 'Улаганский', 'Бай-Тайгинский', 'Барун-Хемчикский', 'Сут-Хольский', 'Шушенский', 'Бейский', 'Аскизский'],
    'Тверь': ['Калининский'],
    'Тимашевский': ['Кореновский', 'Динский', 'Калининский', 'Приморско-Ахтарский', 'Брюховецкий'],
    'Тольятти': ['Жигулевск', 'Ставропольский'],
    'Тотемский': ['Бабушкинский', 'Чухломский', 'Солигаличский', 'Междуреченский', 'Сокольский', 'Сямженский', 'Верховажский', 'Тарногский', 'Нюксенский'],
    'Тула': ['Щекинский', 'Дубенский', 'Алексин', 'Ясногорский', 'Веневский', 'Киреевский', 'Ленинский'],
    'Усть-Джегутинский': ['Усть-Джегутинское'],
    'Учалинский': ['Белорецкий', 'Катав-Ивановский', 'Верхнеуральский', 'Уйский', 'Чебаркульский', 'Миасский', 'Златоустовский'],
    'Хасавюрт': ['Хасавюртовский', 'Новолакский', 'Ахвахский'],
    'Целинский': ['Егорлыкский', 'Зерноградский', 'Сальский', 'Песчанокопский', 'Белоглинский'],
    'Чернский': ['Мценский', 'Корсаковский', 'Каменский', 'Тепло-Огаревский', 'Плавский', 'Арсеньевский', 'Славный'],
    'Ярославль': ['Ярославский'],
     'Советский': ['Будённовский', 'Степновский', 'Кировский', 'Георгиевский', 'Новоселицкий'], 
     'Ульяновский': ['Ульяновск', 'Чердаклинский', 'Новоульяновск', 'Сенгилеевский', 'Теренгульский', 'Майнский', 'Цильнинский', 'Тетюшский'],
     "Владивостокский": ["Хасанский", "Надеждинский", "Артемовский", "Шкотовский", "Большой Камень", "Фокино"],
    "Воротынский": ["Пильнинский", "Спасский", "Лысковский", "Воскресенский", "Юринский", "Горномарийский", "Ядринский"],
    "Джейрах": ["Итум-Калинский", "Ачхой-Мартановский", "Сунженский", "Пригородный", "Владикавказ"],
    "Кизилюртовский": ["Кумторкалинский", "Хунзахский", "Буйнакский", "Казбековский", "Хасавюртовский",
                       "Кизилюрт", "Цумадинский", "Бабаюртовский", "Ахвахский"],
    "Кичменгско-Городецкий": ["Нюксенский", "Бабушкинский", "Никольский", "Вохомский", "Опаринский", "Подосиновский", "Великоустюгский"],
    "Красногвардейский": ["Усть-Лабинский", "Шовгеновский", "Белореченский", "Теучежский", "Краснодар", "Динский"],
    "Красноярск": ["Березовский", "Дивногорск", "Емельяновский"],
    "Нестеровский": ["Краснознаменский", "Гусевский", "Озерский"],
    "Сегежский": ["Беломорский", "Муезерский", "Медвежьегорский", "Онежский"],
    "Ульяновский": ["Ульяновск", "Цильнинский", "Майнский", "Тереньгульский",
                    "Сенгилеевский", "Новоульяновск", "Чердаклинский", "Тетюшский"],
    "Хасавюртовский": ["Хасавюрт", "Новолакский", "Ахвахский", "Гудермесский", "Шелковский",
                       "Бабаюртовский", "Хунзахский", "Кизилюртовский", "Кизилюрт", "Казбековский"],
    "Чайковский": ["Еловский", "Куединский", "Янаульский", "Нефтекамск", "Камбарский",
                   "Сарапульский", "Завьяловский", "Воткинский"],
    "Шовгеновский": ["Курганинский", "Кошехабльский", "Гиагинский", "Белореченский", "Красногвардейский", "Усть-Лабинский"],
    "Южно-Сахалинск": ["Корсаковский", "Анивский", "Холмский", "Долинский"]
}


excluded_pairs = set()
for protest, events in exclude_dict.items():
    for event in events:
        excluded_pairs.add((protest, event))

# Проверка на вырожденные признаки
combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
variances = combined_all.var()
threshold = 1e-8
degenerate = variances[variances <= threshold].index.tolist()
if degenerate:
    print(f"Удаляем вырожденные признаки: {degenerate}")
    features = [f for f in features if f not in degenerate]
else:
    print("Вырожденных признаков нет.")

combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
cov_matrix = np.cov(combined_all.T)
cov_inv = np.linalg.inv(cov_matrix)

# Matching внутри каждого региона с проверкой ограничений
matches_list = []

for region in df_protests['region_name'].unique():
    sub_protests = df_protests[df_protests['region_name'] == region].reset_index(drop=True)
    sub_events = df_events[df_events['region_name'] == region].reset_index(drop=True)
    
    # Фильтрация событий: полнота данных + проверка на исключения
    valid_events = []
    for _, event_row in sub_events.iterrows():
        event_name = event_row['municipality']
        oktmo = event_row['oktmo']
        
        # Проверка полноты данных
        has_full_data = check_years_completeness(oktmo, df_events_full, required_years)
        
        # Проверка, что нет ни одного протеста, для которого это событие запрещено
        is_allowed = all(
            (protest_name, event_name) not in excluded_pairs
            for protest_name in sub_protests['municipality']
        )
        
        if has_full_data and is_allowed:
            valid_events.append(event_row)
    
    if not valid_events:
        print(f"В регионе {region} нет подходящих муниципалитетов событий")
        continue
    
    sub_events_valid = pd.DataFrame(valid_events).reset_index(drop=True)
    
    n_p = sub_protests.shape[0]
    n_e = sub_events_valid.shape[0]
    
    if n_e == 0:
        continue
    
    dist_mat = np.zeros((n_p, n_e))
    
    # Заполнение матрицы расстояний
    for i in range(n_p):
        x = sub_protests.loc[i, features].values
        for j in range(n_e):
            y = sub_events_valid.loc[j, features].values
            dist_mat[i, j] = mahalanobis(x, y, cov_inv)
    
    # Венгерский алгоритм
    row_ind, col_ind = linear_sum_assignment(dist_mat)
    
    # Сбор результатов
    matched_p = sub_protests.iloc[row_ind].reset_index(drop=True)
    matched_e = sub_events_valid.iloc[col_ind].reset_index(drop=True)
    
    matched = matched_p.join(
        matched_e,
        lsuffix="_protest",
        rsuffix="_event"
    )
    matched["mahalanobis_distance"] = dist_mat[row_ind, col_ind]
    matches_list.append(matched)




df_matches = pd.concat(matches_list, axis=0).reset_index(drop=True)
df_matches.to_csv('прототип_данных_2012t+.csv', index=False)
df_matches

Вырожденных признаков нет.
В регионе Чеченская нет подходящих муниципалитетов событий
В регионе Магаданская нет подходящих муниципалитетов событий


,region_name_protest,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,ПотреблениеЭ_protest,ПотреблениеВ_protest,Бюджет_protest,...,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,ПотреблениеЭ_event,ПотреблениеВ_event,Бюджет_event,БезДорог_event,Земля_event,Квартиры_event,mahalanobis_distance
0,Тверская,Андреапольский,28602000,20.4,0.00,0.0,400.0,953.00,5.30,56587.0,...,0.00,4600.0,330.0,993.30,0.00,19706.5,3.48,167100.00,636.54,1.214251
1,Тверская,Селижаровский,28650000,24.2,0.00,100.0,1320.0,1328.00,0.00,51706.0,...,5.73,0.0,3350.0,1453.60,0.00,16843.0,8.30,117998.00,936.40,1.470334
2,Тверская,Тверь,28701000,76.1,0.00,0.0,300.0,692.30,34.90,-427870.0,...,90.00,2400.0,8373.0,580.00,40.00,-58584.0,5.00,133976.00,543.73,3.353463
3,Тверская,Торопецкий,28655000,29.1,0.00,0.0,1400.0,702.00,0.00,-1776.0,...,0.00,200.0,300.0,607.00,0.00,2900.0,2.60,162549.00,0.00,0.457416
4,Тверская,Торжокский,28654000,4.8,0.00,0.0,4260.0,684.00,37.30,8640.0,...,0.00,0.0,477.0,765.00,34.00,6240.0,3.70,300500.00,0.00,1.222320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,Ростовская,Кашарский,60624000,13.8,46.42,0.0,500.0,528.00,0.00,-1934.0,...,68.97,0.0,0.0,522.80,0.00,5104.0,0.00,136084.00,0.00,0.190260
135,Челябинская,Кизильский,75632000,0.4,0.00,0.0,8600.0,1033.07,0.00,9358.0,...,0.00,0.0,8970.0,706.01,0.00,-7285.0,10.16,263401.00,0.00,2.063163
136,Ханты-Мансийский,Мегион,71873000,93.3,0.00,0.0,0.0,1433.40,20.75,-363752.0,...,94.93,0.0,0.0,993.50,18.70,-363931.0,0.00,16890.00,0.00,1.784134
137,Ханты-Мансийский,Нягань,71879000,211.7,88.52,0.0,1600.0,1111.43,19.75,-65228.0,...,0.00,0.0,0.0,998.89,18.01,-76391.0,0.00,4515.23,0.00,0.466402


In [49]:
df_2012 = pd.read_csv('прототип_данных_2012t+.csv')
df_2011 = pd.read_csv('без_протестов2011_2012t+.csv')

df_2012 = df_2012[df_2012["mahalanobis_distance"] <= 10].reset_index(drop=True)

merged_df = pd.merge(
    df_2012,
    df_2011,
    left_on=['municipality_protest', 'oktmo_protest'],
    right_on=['municipality', 'oktmo'],
    how='inner'
)

from2012 = merged_df[[
    'municipality_protest', 'oktmo_protest',
    'Жилье (1000 м2)_protest', 'Дороги (%)_protest',
    'Канализация (1 м)_protest', 'Водопровод (1 метр)_protest',
    'municipality_event', 'oktmo_event',
    'Жилье (1000 м2)_event', 'Дороги (%)_event',
    'Канализация (1 м)_event', 'Водопровод (1 метр)_event', 
]]


from2012.to_csv('связка_махаланобис_2012t+.csv', index=False)
from2012

,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event
0,Торопецкий,28655000,29.1,0.00,0.0,1400.0,Жарковский,28614000,8.0,0.00,200.0,300.0
1,Торжокский,28654000,4.8,0.00,0.0,4260.0,Старицкий,28653000,76.1,0.00,0.0,477.0
2,Холмогорский,11656000,90.0,0.00,620.0,1601.0,Коношский,11622000,73.4,0.00,40.0,3000.0
3,Аургазинский,80605000,2.1,12.00,0.0,30100.0,Шаранский,80658000,0.1,90.60,0.0,8490.0
4,Янаульский,80659000,20.0,14.20,0.0,5800.0,Ишимбайский,80631000,15.8,67.20,0.0,7700.0
...,...,...,...,...,...,...,...,...,...,...,...,...
85,Кашарский,60624000,13.8,46.42,0.0,500.0,Волгодонской,60612000,2.6,68.97,0.0,0.0
86,Кизильский,75632000,0.4,0.00,0.0,8600.0,Уйский,75656000,25.7,0.00,0.0,8970.0
87,Мегион,71873000,93.3,0.00,0.0,0.0,Радужный,71877000,12.3,94.93,0.0,0.0
88,Нягань,71879000,211.7,88.52,0.0,1600.0,Покачи,71884000,6.6,0.00,0.0,0.0


In [ ]:
df = pd.read_csv('общие_данные_2012t+.csv')

df = df[['year', 'municipality', 'oktmo', 'Жилье (1000 м2)', 'Дороги (%)', 'Канализация (1 м)', 'Водопровод (1 метр)', 
         'region_name',
        ]]


df = df.drop_duplicates(subset=['year', 'municipality', 'oktmo'])
df = df[df['year'].isin([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])]



events = df
from2012 = pd.read_csv('связка_махаланобис_2012t+.csv')
final = pd.DataFrame()

for _, link_row in from2012.iterrows():
    
    protest_filter = (
        (events['municipality'] == link_row['municipality_protest']) &
        (events['oktmo'] == link_row['oktmo_protest'])
    )
    protest_data = events[protest_filter].add_suffix('_protest')
    
    event_filter = (
        (events['municipality'] == link_row['municipality_event']) &
        (events['oktmo'] == link_row['oktmo_event'])
    )
    event_data = events[event_filter].add_suffix('_event')
    
    merged = protest_data.merge(
        event_data,
        left_on='year_protest',
        right_on='year_event',
        how='left'
    ).rename(columns={'year_protest': 'year'})
    

    merged = merged.drop('year_event', axis=1)
    final = pd.concat([final, merged], ignore_index=True)


final.to_csv('2012t+.csv', index=False)
final


,year,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,region_name_event
0,2008,Торопецкий,28655000,24.0,0.00,300.0,500.0,Тверская,Жарковский,28614000.0,6.82,0.00,80.0,100.0,Тверская
1,2009,Торопецкий,28655000,29.0,0.00,0.0,0.0,Тверская,Жарковский,28614000.0,7.94,0.00,0.0,636.0,Тверская
2,2010,Торопецкий,28655000,29.1,0.00,0.0,0.0,Тверская,Жарковский,28614000.0,8.20,0.00,200.0,350.0,Тверская
3,2011,Торопецкий,28655000,29.1,0.00,0.0,500.0,Тверская,Жарковский,28614000.0,7.90,0.00,0.0,200.0,Тверская
4,2012,Торопецкий,28655000,29.1,0.00,0.0,1400.0,Тверская,Жарковский,28614000.0,8.00,0.00,200.0,300.0,Тверская
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1149,2016,Солигаличский,34640000,0.0,38.09,0.0,1.0,Костромская,Вохомский,34606000.0,0.00,13.47,0.0,7600.0,Костромская
1150,2017,Солигаличский,34640000,0.0,38.52,0.0,351.0,Костромская,Вохомский,34606000.0,0.00,13.97,0.0,280.0,Костромская
1151,2018,Солигаличский,34640000,0.0,38.52,0.0,122.0,Костромская,Вохомский,34606000.0,0.00,14.18,0.0,7000.0,Костромская
1152,2019,Солигаличский,34640000,0.0,38.71,0.0,545.0,Костромская,Вохомский,34606000.0,0.00,14.21,0.0,700.0,Костромская
